In [1]:
import torch
import torch.nn as nn

In [18]:
from modeling_gemma import PaliGemmaForConditionalGeneration, PaliGemmaConfig
from transformers import AutoTokenizer
import json
import glob
from safetensors import safe_open
from typing import Tuple
import os
from PIL import Image
import numpy as np

In [19]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
from datasets import load_dataset

ds = load_dataset("beans", split="train")
print(len(ds))

Using the latest cached version of the dataset since beans couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/htkumar/.cache/huggingface/datasets/beans/default/0.0.0/27aa014ce09b193e1a6f58112d4a66e0eddb69c5 (last modified on Sat Sep 14 14:21:10 2024).


1034


In [22]:
ds[1000]['image'].save('/home/htkumar/llms/paligemma-pytorch/beans_1000_leaf.jpeg')

In [23]:
image = Image.open('/home/htkumar/llms/paligemma-pytorch/beans_1000_leaf.jpeg')
np.array(image).shape

(500, 500, 3)

In [ ]:
from datasets import load_dataset, Image
dataset = load_dataset("ILSVRC/imagenet-1k", split="train")

In [ ]:
model_path = "/home/htkumar/paligemma-3b-pt-224"
device = 'cuda'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
assert tokenizer.padding_side == "right"

In [ ]:
tokenizer.encode("How are you?")

In [ ]:
tokenizer.max_len_single_sentence

In [ ]:
safetensor_files = glob.glob(os.path.join(model_path, "*.safetensors"))
safetensor_files

In [ ]:
tensors = {}
for safetensors_file in safetensor_files:
    with safe_open(safetensors_file, framework="pt", device='cpu') as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)

In [ ]:
len(tensors)

In [ ]:
# Load the model config
with open(os.path.join(model_path, "config.json"), "r") as f:
    model_config_file = json.load(f)
    config = PaliGemmaConfig(**model_config_file)

In [ ]:
print(config)

In [ ]:
model = PaliGemmaForConditionalGeneration(config).to(device)

In [ ]:
model.load_state_dict(tensors, strict=False)

In [ ]:
# model.load_state_dict??

In [ ]:
model.tie_weights()

In [ ]:
model.language_model.model.embed_tokens.weight.shape

In [ ]:
target = torch.tensor([1, 2, 3, 4, 5])
mask = torch.tensor([False, True, False, True, False])
source = torch.tensor([10, 20])

result = target.masked_scatter(mask, source)
result

In [ ]:
attention_mask = torch.full(
    (4, 12), fill_value=1
)
attention_mask.shape

In [ ]:
attention_mask.cumsum(-1)[:, -1].dim()

In [ ]:
(attention_mask.cumsum(-1)).masked_fill_((attention_mask == 0), 1)

In [ ]:
vocab_size, hidden_state = 10000, 256
lm_head = nn.Linear(hidden_state, vocab_size)
embed_tokens = nn.Embedding(vocab_size, hidden_state)
lm_head.weight.shape, embed_tokens.weight.shape

In [ ]:
torch.equal(lm_head.weight, embed_tokens.weight)

In [ ]:
lm_head.weight = embed_tokens.weight

In [ ]:
torch.equal(lm_head.weight, embed_tokens.weight)

In [ ]:
base, dim = 1000, 128
inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
inv_freq.shape, inv_freq.dtype

In [ ]:
inv_freq_expanded = inv_freq[None, :, None]
inv_freq_expanded.shape

In [ ]:
bs, seq_len, n_heads = 2, 10, 4
position_ids = torch.cat([torch.arange(seq_len)[None, :] for _ in range(bs)], dim=0)
position_ids

In [ ]:
position_ids_expanded = position_ids[:, None, :].float()
position_ids_expanded.shape

In [ ]:
freqs = (inv_freq_expanded @ position_ids_expanded).transpose(1, 2)
freqs.shape

In [ ]:
freqs

In [ ]:
q = torch.ones(bs, n_heads, seq_len, dim)
q.shape

In [ ]:
x1 = q[..., :q.shape[-1] // 2]
x2 = q[..., q.shape[-1] // 2:]
x1.shape, x2.shape

In [ ]:
a = torch.cat((-x2, x1), dim=-1)
a.shape

In [ ]:
a

In [ ]:
freqs.shape

In [ ]:
emb = torch.cat((freqs, freqs), dim=-1)
emb.shape

In [ ]:
cos = emb.cos()
cos.shape

In [ ]:
cos = cos.unsqueeze(1)
cos.shape

In [ ]:
a.shape

In [ ]:
def rotate_half(x):
    x1 = x[..., :x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2:]
    return torch.cat((-x2, x1), dim=-1)

In [ ]:
x = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]])
x

In [ ]:
x.shape

In [ ]:
rotate_half(x)

In [ ]:
a = torch.arange(10).expand(1, -1)
a.shape

In [ ]:
num_positions = 100
emb_dim = 128
B = 4

In [ ]:
emb_layer = nn.Embedding(num_positions, emb_dim)

In [ ]:
conv_output = torch.randn(B, num_positions, emb_dim)

In [ ]:
position_ids = torch.arange(num_positions)
position_emb = emb_layer(position_ids)
position_emb.shape

In [ ]:
out_1 = conv_output + position_emb

In [ ]:
out_2 = conv_output + emb_layer(position_ids.expand(1, -1))
out_2.shape

In [ ]:
torch.equal(out_1, out_2)

In [ ]:
B, num_heads, seq_len, emb_dim = 4, 2, 10, 128

In [ ]:
attn_scores = torch.randn(B, num_heads, seq_len, seq_len)
values = torch.randn(B, num_heads, seq_len, emb_dim // num_heads)
attn_scores.shape, values.shape

In [ ]:
out = attn_scores @ values
out.shape

In [ ]:
act = torch.ones(B, seq_len, emb_dim)
act.shape

In [ ]:
act.mean(-1, keepdim=True).shape